Data Source: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

# Step 1

In [15]:
import pdb
import pandas as pd
import multiprocessing
from multiprocessing import Pool, Lock, Process, Manager
from tqdm import tqdm
import csv
from pandas import read_csv

In [16]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [17]:
def add_dict(window,total):
        if bool(dictionary[window]) == True:
                value = dictionary[window]
                value = float(value) + float(total)
                dictionary[window] = value
        if bool(dictionary[window]) == False:
                dictionary[window] = total

In [18]:
def add_dict2(ID,window,total):
        if bool(dictionary2[ID][window]) == True:
                value = dictionary2[ID][window]
                value = float(value) + float(total)
                dictionary2[ID][window] = value
        if bool(dictionary2[ID][window]) == False:
                dictionary2[ID][window] = total

In [19]:
def add_dict3(ID,window,total):
        if bool(dictionary3[ID][window]) == True:
                value = dictionary3[ID][window]
                value = float(value) + float(total)
                dictionary3[ID][window] = value
        if bool(dictionary3[ID][window]) == False:
                dictionary3[ID][window] = total

# Step 2

In [20]:
def extract_info(file_1):
    """
    param: file.csv
    return: dataframe
    This function takes as input a file and stores usefule information into a dataframe
    """
    data = pd.read_csv(file_1)
    data = data[['VendorID','tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID','passenger_count','trip_distance','total_amount']].copy()
    return data
    

In [21]:
"""
Data files for yellow taxi cab (2019)
"""

taxi_01_2019 = extract_info('yellow_tripdata_2019-01.csv')
taxi_02_2019 = extract_info('yellow_tripdata_2019-02.csv')
taxi_03_2019 = extract_info('yellow_tripdata_2019-03.csv')
taxi_04_2019 = extract_info('yellow_tripdata_2019-04.csv')
taxi_05_2019 = extract_info('yellow_tripdata_2019-05.csv')
taxi_06_2019 = extract_info('yellow_tripdata_2019-06.csv')


# Step 3

In [22]:
n = [taxi_01_2019,taxi_02_2019,taxi_03_2019,taxi_04_2019,taxi_05_2019,taxi_06_2019] #information for 2019
taxi_2019 = pd.concat(n,ignore_index=True)

In [31]:
dictionary = AutoVivification()
print(len(taxi_2019['tpep_pickup_datetime']))
for index,value in enumerate(taxi_2019['tpep_pickup_datetime']):
    add_dict(taxi_2019['tpep_pickup_datetime'][index].split()[0],1)

44459136


In [32]:
with open('taxi_2019.csv', 'w') as f:
    for key in dictionary.keys():
        f.write("%s,%s\n"%(key,dictionary[key]))

x = read_csv('taxi_2019.csv')
x.columns = ['Date','Number of Rides']
x.to_csv('taxi_2019.csv')

In [23]:
def heap_map(taxi,location):
    """
    param: file.csv
    return: dataframe
    This function takes as input a dataframe and compares features of dataframe to find correlation in data
    """
    taxi_loc = pd.read_csv(location)
    
    data_loc_and_bor = taxi_loc[['LocationID','Borough']]
    
    data_loc_and_bor.columns = ['PULocationID','PUBorough']
    taxi1 = taxi[['PULocationID','tpep_pickup_datetime']]
    taxi1.columns = ['PULocationID','tpep_pickup_datetime']
    taxi1 = data_loc_and_bor.merge(taxi1, on=['PULocationID'], how='right',sort=False) 
    
    data_loc_and_bor.columns = ['DOLocationID','DOBorough']
    taxi2 = taxi[['DOLocationID','tpep_dropoff_datetime']]
    taxi2.columns = ['DOLocationID','tpep_dropoff_datetime']  
    taxi2 = data_loc_and_bor.merge(taxi2, on=['DOLocationID'], how='right',sort=False) 
    
    taxi = pd.concat([taxi1,taxi2],axis = 1,sort=False)
    return taxi
    #use dataframe to find correlation in data
    

In [24]:
taxi_01_2019_heap = heap_map(taxi_01_2019,'../taxi+_zone_lookup.csv')
taxi_02_2019_heap = heap_map(taxi_02_2019,'../taxi+_zone_lookup.csv')
taxi_03_2019_heap = heap_map(taxi_03_2019,'../taxi+_zone_lookup.csv')
taxi_04_2019_heap = heap_map(taxi_04_2019,'../taxi+_zone_lookup.csv')
taxi_05_2019_heap = heap_map(taxi_05_2019,'../taxi+_zone_lookup.csv')
taxi_06_2019_heap = heap_map(taxi_06_2019,'../taxi+_zone_lookup.csv')

In [25]:
n = [taxi_01_2019_heap,taxi_02_2019_heap,taxi_03_2019_heap,taxi_04_2019_heap,taxi_05_2019_heap,taxi_06_2019_heap] #information for 2019
taxi_2019 = pd.concat(n,ignore_index=True)

In [26]:
dictionary2 = AutoVivification()
dictionary3 = AutoVivification()
print(len(taxi_2019['tpep_pickup_datetime']))

for index,value in enumerate(taxi_2019['tpep_pickup_datetime']):
    add_dict2(taxi_2019['PULocationID'][index],taxi_2019['tpep_pickup_datetime'][index].split()[0],1)

for index,value in enumerate(taxi_2019['tpep_dropoff_datetime']) :
    add_dict3(taxi_2019['PULocationID'][index],taxi_2019['tpep_dropoff_datetime'][index].split()[0],1)



44459136


In [27]:
with open('taxi_heap_2019_pick.csv', 'w') as f:
    for key,v1 in dictionary2.items():
        key1 = key
        key2 = list(v1.items())[0][0]
        val = list(v1.items())[0][1]
        f.write("%s,%s,%s\n"%(key1,key2,val))
        
with open('taxi_heap_2019_drop.csv', 'w') as f:
    for key,v1 in dictionary3.items():
        key1 = key
        key2 = list(v1.items())[0][0]
        val = list(v1.items())[0][1]
        f.write("%s,%s,%s\n"%(key1,key2,val))
        
x = read_csv('taxi_heap_2019_pick.csv')
y = read_csv('taxi_heap_2019_drop.csv')
x.columns = ['PULocationID','Date','Number of Rides']
y.columns = ['DOLocationID','Date','Number of Rides']
x.to_csv('taxi_heap_2019_pick.csv')
y.to_csv('taxi_heap_2019_drop.csv')

In [1]:
"""
dictionary2 = AutoVivification()
dictionary3 = AutoVivification()
print(len(taxi_2019['PULocationID']))

for index,value in enumerate(taxi_2019['tpep_pickup_datetime']['PUBorough']):
    add_dict(dictionary2,taxi_2019['tpep_pickup_datetime']['PUBorough'][index].split()[0],1)
    
for index,value in enumerate(taxi_2019['DOBorough']):
    add_dict(dictionary3,taxi_2019['DOBorough'][index].split()[0],1)
"""

"\ndictionary2 = AutoVivification()\ndictionary3 = AutoVivification()\nprint(len(taxi_2019['PULocationID']))\n\nfor index,value in enumerate(taxi_2019['tpep_pickup_datetime']['PUBorough']):\n    add_dict(dictionary2,taxi_2019['tpep_pickup_datetime']['PUBorough'][index].split()[0],1)\n    \nfor index,value in enumerate(taxi_2019['DOBorough']):\n    add_dict(dictionary3,taxi_2019['DOBorough'][index].split()[0],1)\n"